# Setup Environment

In [ ]:
%load_ext autoreload
import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
path_to_add = os.path.join(parent_dir, "src")
sys.path.insert(0, path_to_add)

# Get Data

## Free text comments

In [110]:
import requests
import pandas as pd
import janitor

# URL of the endpoint
url = "https://data.austintexas.gov/resource/jeyv-db9u.json"

# Fetch the data from the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(response.json()).clean_names()
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

## Questions data (likert)

In [ ]:
url_likert = "https://data.austintexas.gov/resource/s2py-ceb7.json"
data_dictionary_url = "https://data.austintexas.gov/api/views/s2py-ceb7/columns.json"

url = "https://data.austintexas.gov/dataset/Community-Survey/s2py-ceb7/about_data"

response = requests.get(url_likert)
if response.status_code == 200:
    # Convert the JSON data to a pandas DataFrame
    dfq = pd.DataFrame(response.json())
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

data_dictionary = requests.get(data_dictionary_url)
if response.status_code == 200:
    data_dictionary = data_dictionary.json()
else:
    print(f"Failed {response.status_code}")

In [ ]:
dfl = pd.read_csv("../data/Community_Survey_20240911.csv")

In [ ]:
likert_columns = ['the_city_of_austin_as_a_place_1', 'the_city_of_austin_as_a_place_2',
       'the_city_of_austin_as_a_place_3', 'overall_quality_of_life_in',]

column_rename = {'the_city_of_austin_as_a_place_1' : 'place to work',
                 'the_city_of_austin_as_a_place_2' : 'place to raise children',
                 'the_city_of_austin_as_a_place_3' : 'place to retire',
                 'overall_quality_of_life_in' : 'overall quality of life in the city'}

dfq = dfq.rename_columns(column_rename)

likert_encoded = ["likert_encoded_" + col for col in column_rename.values()]

In [ ]:
%autoreload 2
from pandas_survey_toolkit import analytics, nlp, vis
from pandas_survey_toolkit.vis import cluster_heatmap_plot


In [ ]:
dfq2 = dfq.cluster_questions(column_rename.values())

In [ ]:
cluster_heatmap_plot(dfq2, x="question_cluster_id", y = likert_encoded)

In [ ]:
dfq.cluster_questions()

In [ ]:
dfq.to_excel('austin_likert.xlsx')

In [126]:
%autoreload 2
from pandas_survey_toolkit import analytics, nlp, vis
from pandas_survey_toolkit.vis import create_keyword_graph, visualize_keyword_graph,visualize_keyword_graph_force, create_keyword_sentiment_df_simple, create_sentiment_color_mapping

In [127]:
df2 = df.extract_keywords("response", top_n=4, ngram_range=(1,1), min_df=5, min_proportion_with_keywords=0.85)

Refinement complete. Min count used: 5
Original average keywords per row: 2.02
Refined average keywords per row: 1.90
Proportion of rows with keywords after refinement: 85.03%
Total unique keywords before refinement: 367
Total unique keywords after refinement: 123
Reduction in unique keywords: 66.49%


In [124]:
df2.refined_keywords.map(lambda x : len(x) > 0).mean()

c:\Users\LONAL23\OneDrive - PA Consulting Group\7 - Data Scientist\pandas-survey-toolkit\src\pandas_survey_toolkit\nlp.py:22: UserWarning: registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'cluster_questions' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  def cluster_questions(df, columns=None, pattern=None, likert_mapping=None,
c:\Users\LONAL23\OneDrive - PA Consulting Group\7 - Data Scientist\pandas-survey-toolkit\src\pandas_survey_toolkit\nlp.py:70: UserWarning: registration of accessor <class 'pandas_flavor.register.register_dataframe_method.<locals>.inner.<locals>.AccessorMethod'> under name 'encode_likert' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  def encode_likert(df, likert_columns, output_prefix='likert_encoded_', custom_mapping=None, debug=True):
c:\Users\LONAL23\OneDriv

0.8503480278422274

In [128]:
df2 = df2.extract_sentiment("response")

c:\Users\LONAL23\OneDrive - PA Consulting Group\7 - Data Scientist\pandas-survey-toolkit\.venv\Lib\site-packages\transformers\modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this ex

In [ ]:
import datamapplot

In [ ]:
def collect_unique_keywords(df, group_column, keyword_column):
    # Define a function to convert list to set and combine sets
    def combine_keywords(series):
        return set().union(*[set(keywords) for keywords in series if isinstance(keywords, list)])
    
    # Group by the specified column and aggregate keywords
    grouped = df.groupby(group_column)[keyword_column].agg(combine_keywords).reset_index()
    
    # Rename the aggregated column
    grouped = grouped.rename(columns={keyword_column: f'unique_{keyword_column}'})
    
    return grouped

df_labels = collect_unique_keywords(df2, "cluster", "refined_keywords")

In [ ]:
df2 = df2.merge(df_labels, on="cluster")

In [ ]:
df2["labels"] = df2["unique_refined_keywords"].map(lambda x: "-".join(x))

In [ ]:
import numpy as np


datamapplot.create_plot(df2[['umap_x', 'umap_y']].values, df2["labels"].values,)# hover_text =df2["response"].values )

In [129]:
sentiment_df = create_keyword_sentiment_df_simple(df2)
sentiment_df = sentiment_df.query("word != 'not'")
color_mapping = create_sentiment_color_mapping(sentiment_df)

G = create_keyword_graph(df2, 'keywords', node_color_mapping=color_mapping)
G.remove_node("not")
visualize_keyword_graph_force(G, output_file='keyword_sentiment_graph.html', colormap='RdBu', min_edge_count=2, min_node_count=1)

Graph saved to keyword_sentiment_graph.html
